In [1]:
!pip install scikit-learn==0.20.3 --upgrade
!pip install xgboost==0.71 --upgrade

Requirement already up-to-date: scikit-learn==0.20.3 in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.20.3)
     |████████████████████████████████| 501kB 8.4MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/4e/6d/1d/0bc23240225fe411315d8abb5d4521b9ff002493ff77515ccc
Successfully built xgboost


In [2]:
# Em seguida iremos importar diversas bibliotecas que serão utilizadas:

# Pacote para trabalhar com JSON
import json

# Pacote para realizar requisições HTTP
import requests

# Pacote para exploração e análise de dados
import pandas as pd

# Pacote com métodos numéricos e representações matriciais
import numpy as np

# Pacote para construção de modelo baseado na técnica Gradient Boosting
import xgboost as xgb

# Pacotes do scikit-learn para pré-processamento de dados
# "SimpleImputer" é uma transformação para preencher valores faltantes em conjuntos de dados
from sklearn.impute import SimpleImputer

# Pacotes do scikit-learn para treinamento de modelos e construção de pipelines
# Método para separação de conjunto de dados em amostras de treino e teste
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
# Método para criação de modelos baseados em árvores de decisão
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# Classe para a criação de uma pipeline de machine-learning
from sklearn.pipeline import Pipeline

# Pacotes do scikit-learn para avaliação de modelos
# Métodos para validação cruzada do modelo criado
from sklearn.model_selection import KFold, cross_validate

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_9386c01e35e043869adf93baf404f5b0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='GGLmaafItps2dpRz1Knp25uJcMB8TUBzftKCs85kJ6cH',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_9386c01e35e043869adf93baf404f5b0.get_object(Bucket='behindthecode02-donotdelete-pr-m3mvumkv1ilmri',Key='dataset_desafio_2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


MATRICULA                       NOME  REPROVACOES_DE  REPROVACOES_EM  \
0     502375          Márcia Illiglener               0               0   
1     397093   Jason Jytereoman Izoimum               0               0   
2     915288  Bartolomeu Inácio da Gama               0               0   
3     192652            Fernanda Guedes               1               3   
4     949491     Alessandre Borba Gomes               1               3   

   REPROVACOES_MF  REPROVACOES_GO  NOTA_DE  NOTA_EM  NOTA_MF  NOTA_GO  INGLES  \
0               0               0      6.2      5.8      4.6      5.9     0.0   
1               0               0      6.0      6.2      5.2      4.5     1.0   
2               0               0      7.3      6.7      7.1      7.2     0.0   
3               1               1      0.0      0.0      0.0      0.0     1.0   
4               1               1      0.0      0.0      0.0      0.0     1.0   

   H_AULA_PRES  TAREFAS_ONLINE  FALTAS       PERFIL  
0            2               4       3       EXATAS  
1            2               4       3       EXATAS  
2            5               0       3      HUMANAS  
3            4               4       4  DIFICULDADE  
4            5               2       5  DIFICULDADE

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin


# All sklearn Transforms must have the `transform` and `fit` methods
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Primeiro realizamos a cópia do dataframe 'X' de entrada
        data = X.copy()
        # Retornamos um novo dataframe sem as colunas indesejadas
        return data.drop(labels=self.columns, axis='columns')

In [6]:
# Instanciando uma transformação DropColumns
rm_columns = DropColumns(
    columns=["NOME", "MATRICULA", "INGLES"]  # Essa transformação recebe como parâmetro uma lista com os nomes das colunas indesejadas
)

print(rm_columns)

DropColumns(columns=['NOME', 'MATRICULA', 'INGLES'])


In [7]:
# Aplicando a transformação ``DropColumns`` ao conjunto de dados base
rm_columns.fit(X=df_data_1)

# Reconstruindo um DataFrame Pandas com o resultado da transformação
df_data_2 = pd.DataFrame.from_records(
    data=rm_columns.transform(
        X=df_data_1
    ),
)

In [8]:
# Visualizando as colunas do dataset transformado
print("Colunas do dataset após a transformação ``DropColumns``: \n")
print(df_data_2.columns)

Colunas do dataset após a transformação ``DropColumns``: 

Index(['REPROVACOES_DE', 'REPROVACOES_EM', 'REPROVACOES_MF', 'REPROVACOES_GO',
       'NOTA_DE', 'NOTA_EM', 'NOTA_MF', 'NOTA_GO', 'H_AULA_PRES',
       'TAREFAS_ONLINE', 'FALTAS', 'PERFIL'],
      dtype='object')


In [9]:
# Criação de um objeto ``SimpleImputer``
si = SimpleImputer(
    missing_values=np.nan,  # os valores faltantes são do tipo ``np.nan`` (padrão Pandas)
    strategy='constant',  # a estratégia escolhida é a alteração do valor faltante por uma constante
    fill_value=0,  # a constante que será usada para preenchimento dos valores faltantes é um int64=0.
    verbose=0,
    copy=True
)

In [10]:
# Aplicamos o SimpleImputer ``si`` ao conjunto de dados df_data_2 (resultado da primeira transformação)
si.fit(X=df_data_2)

# Reconstrução de um novo DataFrame Pandas com o conjunto imputado (df_data_3)
df_data_3 = pd.DataFrame.from_records(
    data=si.transform(
        X=df_data_2
    ),  # o resultado SimpleImputer.transform(<<pandas dataframe>>) é lista de listas
    columns=df_data_2.columns  # as colunas originais devem ser conservadas nessa transformação
)

In [11]:
# Visualizando os dados faltantes do dataset após a segunda transformação (SimpleImputer) (df_data_3)
print("Valores nulos no dataset após a transformação SimpleImputer: \n\n{}\n".format(df_data_3.isnull().sum(axis = 0)))

Valores nulos no dataset após a transformação SimpleImputer: 

REPROVACOES_DE    0
REPROVACOES_EM    0
REPROVACOES_MF    0
REPROVACOES_GO    0
NOTA_DE           0
NOTA_EM           0
NOTA_MF           0
NOTA_GO           0
H_AULA_PRES       0
TAREFAS_ONLINE    0
FALTAS            0
PERFIL            0
dtype: int64



In [12]:
# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_data_3[features]
y = df_data_3[target]

In [27]:
X.head()

REPROVACOES_DE  REPROVACOES_EM  REPROVACOES_MF  REPROVACOES_GO  NOTA_DE  \
0               0               0               0               0      6.2   
1               0               0               0               0      6.0   
2               0               0               0               0      7.3   
3               1               3               1               1      0.0   
4               1               3               1               1      0.0   

   NOTA_EM  NOTA_MF  NOTA_GO  H_AULA_PRES  TAREFAS_ONLINE  FALTAS  
0      5.8      4.6      5.9            2               4       3  
1      6.2      5.2      4.5            2               4       3  
2      6.7      7.1      7.2            5               0       3  
3      0.0      0.0      0.0            4               4       4  
4      0.0      0.0      0.0            5               2       5

In [30]:
y_array = np.ravel(y)

In [31]:
classifier_rf = RandomForestClassifier()

In [35]:
param_grid = {
            "criterion": ['entropy', 'gini'],
            "n_estimators": [50, 75, 100],
            "max_depth": [5, 10, 15],
}
grid_search = GridSearchCV(classifier_rf, param_grid, scoring="accuracy")
grid_search.fit(X, y_array)

classifier_rf = grid_search.best_estimator_ 

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [38]:
scores_rf = cross_val_score(classifier_rf, X, y_array,
                            scoring='accuracy', cv=5)

print(scores_rf.mean())

0.8139493101978967


In [36]:
grid_search.best_params_, grid_search.best_score_

({'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 100}, 0.81355)